# 84. 総合比較と推奨

## 目的
- 実験80-83の結果を統合し、全パイプラインを統一基準で比較
- Pareto最適フロンティア（Recall vs 削減率）を特定

## 比較パイプライン
| ID | パイプライン | 説明 |
|----|-------------|------|
| A | ITQ Baseline | ITQ → Hamming top-K → Cosine |
| B | ITQ + Pivot | ITQ → Pivot → Hamming → Cosine |
| C | ITQ + Band | ITQ → Band filter → Hamming → Cosine |
| D | ITQ + Band + Pivot | ITQ → Band → Pivot → Hamming → Cosine |
| E | ITQ + Confidence probe | ITQ → Confidence bands → Hamming → Cosine |
| F | ITQ + Confidence + Pivot | ITQ → Confidence bands → Pivot → Hamming → Cosine |

## 0. セットアップ

In [1]:
import sys
import numpy as np
import time
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, '../src')
from itq_lsh import ITQLSH, hamming_distance, hamming_distance_batch
from dflsh import build_band_index, band_filter, confidence_multiprobe

DATA_DIR = Path('../data')
np.random.seed(42)

N_QUERIES = 100
TOP_K = 10
print(f'Configuration: N_QUERIES={N_QUERIES}, TOP_K={TOP_K}')

Configuration: N_QUERIES=100, TOP_K=10


## 1. データロード

In [2]:
datasets = {}

# English E5-base
datasets['EN'] = {
    'embeddings': np.load(DATA_DIR / '10k_e5_base_en_embeddings.npy'),
    'hashes': np.load(DATA_DIR / '10k_e5_base_en_hashes_128bits.npy'),
    'pivot_dist': np.load(DATA_DIR / '10k_e5_base_en_pivot_distances.npy'),
    'pivots': np.load(DATA_DIR / 'pivots_8_e5_base_en.npy'),
    'itq_model': ITQLSH.load(str(DATA_DIR / 'itq_e5_base_128bits.pkl')),
}

# Japanese E5-base
datasets['JA'] = {
    'embeddings': np.load(DATA_DIR / '10k_e5_base_ja_embeddings.npy'),
    'hashes': np.load(DATA_DIR / '10k_e5_base_ja_hashes_128bits.npy'),
    'pivot_dist': np.load(DATA_DIR / '10k_e5_base_ja_pivot_distances.npy'),
    'pivots': np.load(DATA_DIR / 'pivots_8_e5_base_ja.npy'),
    'itq_model': ITQLSH.load(str(DATA_DIR / 'itq_e5_base_128bits.pkl')),
}

# MiniLM
datasets['MiniLM'] = {
    'embeddings': np.load(DATA_DIR / '10k_minilm_embeddings.npy'),
    'hashes': np.load(DATA_DIR / '10k_minilm_hashes_128bits.npy'),
    'pivot_dist': np.load(DATA_DIR / '10k_minilm_pivot_distances.npy'),
    'pivots': np.load(DATA_DIR / 'pivots_8_minilm.npy'),
    'itq_model': ITQLSH.load(str(DATA_DIR / 'itq_minilm_128bits.pkl')),
}

for name, d in datasets.items():
    print(f'{name}: emb={d["embeddings"].shape}, hash={d["hashes"].shape}, '
          f'pivots={d["pivots"].shape}')

EN: emb=(10000, 768), hash=(10000, 128), pivots=(8, 128)
JA: emb=(10000, 768), hash=(10000, 128), pivots=(8, 128)
MiniLM: emb=(10000, 384), hash=(10000, 128), pivots=(8, 128)


## 2. 統一評価関数

In [3]:
def get_ground_truth(embeddings, qi, top_k=10):
    cos_sims = cosine_similarity(embeddings[qi:qi+1], embeddings)[0]
    cos_sims[qi] = -1
    return set(np.argsort(cos_sims)[-top_k:])


def evaluate_pipeline_unified(
    embeddings, hashes, pivot_distances, pivots, itq_model,
    pipeline_id, pipeline_name,
    use_band=False, band_width=8, min_band_matches=1,
    use_pivot=False, pivot_threshold=20,
    use_confidence=False, max_probes=0,
    candidate_limit=500
):
    """統一パイプライン評価関数"""
    rng = np.random.default_rng(42)
    query_indices = rng.choice(len(embeddings), N_QUERIES, replace=False)
    
    bi = build_band_index(hashes, band_width) if (use_band or use_confidence) else None
    all_projections = None
    if use_confidence:
        _, all_projections = itq_model.transform_with_confidence(embeddings)
    
    filter_recalls = []
    final_recalls = []
    filter_counts = []
    total_times = []
    
    for qi in query_indices:
        gt = get_ground_truth(embeddings, qi, TOP_K)
        query_hash = hashes[qi]
        
        start = time.time()
        
        # Stage 1: Candidate selection
        if use_confidence:
            query_proj = all_projections[qi]
            cands = confidence_multiprobe(
                query_hash, query_proj, bi, band_width,
                max_probes=max_probes, order='confidence'
            )
            cands = cands[cands != qi]
        elif use_band:
            cands = band_filter(query_hash, bi, band_width, min_matches=min_band_matches)
            cands = cands[cands != qi]
        else:
            cands = np.arange(len(embeddings))
            cands = cands[cands != qi]
        
        # Stage 2: Pivot filter
        if use_pivot and len(cands) > 0:
            query_pivot_dists = np.array([
                hamming_distance(query_hash, p) for p in pivots
            ])
            cand_pivot_dists = pivot_distances[cands]
            mask = np.ones(len(cands), dtype=bool)
            for i in range(len(pivots)):
                lower = query_pivot_dists[i] - pivot_threshold
                upper = query_pivot_dists[i] + pivot_threshold
                mask &= (cand_pivot_dists[:, i] >= lower) & (cand_pivot_dists[:, i] <= upper)
            cands = cands[mask]
        
        filter_counts.append(len(cands))
        filter_recalls.append(len(gt & set(cands)) / TOP_K)
        
        # Stage 3: Hamming sort + Cosine rerank
        if len(cands) > 0:
            h_dists = hamming_distance_batch(query_hash, hashes[cands])
            top_idx = np.argsort(h_dists)[:candidate_limit]
            final_cands = cands[top_idx]
            
            cand_cos = cosine_similarity(embeddings[qi:qi+1], embeddings[final_cands])[0]
            top_in_cand = final_cands[np.argsort(cand_cos)[-TOP_K:]]
            final_recalls.append(len(gt & set(top_in_cand)) / TOP_K)
        else:
            final_recalls.append(0.0)
        
        total_times.append(time.time() - start)
    
    return {
        'id': pipeline_id,
        'name': pipeline_name,
        'filter_candidates': np.mean(filter_counts),
        'filter_candidates_std': np.std(filter_counts),
        'reduction': 1 - np.mean(filter_counts) / len(embeddings),
        'filter_recall': np.mean(filter_recalls),
        'recall_at_k': np.mean(final_recalls),
        'time_ms': np.mean(total_times) * 1000,
    }

## 3. 全パイプライン評価

In [4]:
def run_all_pipelines(data, dataset_name, candidate_limit=500):
    """全パイプラインを評価"""
    emb = data['embeddings']
    hashes = data['hashes']
    pd = data['pivot_dist']
    pivots = data['pivots']
    itq_model = data['itq_model']
    
    results = []
    
    # A: ITQ Baseline (no filter)
    results.append(evaluate_pipeline_unified(
        emb, hashes, pd, pivots, itq_model,
        'A', 'ITQ Baseline',
        candidate_limit=candidate_limit
    ))
    
    # B: ITQ + Pivot (threshold=20)
    results.append(evaluate_pipeline_unified(
        emb, hashes, pd, pivots, itq_model,
        'B', 'ITQ+Pivot(t=20)',
        use_pivot=True, pivot_threshold=20,
        candidate_limit=candidate_limit
    ))
    
    # B2: ITQ + Pivot (threshold=25)
    results.append(evaluate_pipeline_unified(
        emb, hashes, pd, pivots, itq_model,
        'B2', 'ITQ+Pivot(t=25)',
        use_pivot=True, pivot_threshold=25,
        candidate_limit=candidate_limit
    ))
    
    # C: ITQ + Band (bw=8)
    results.append(evaluate_pipeline_unified(
        emb, hashes, pd, pivots, itq_model,
        'C', 'ITQ+Band(bw=8)',
        use_band=True, band_width=8,
        candidate_limit=candidate_limit
    ))
    
    # D: ITQ + Band + Pivot (bw=8, pt=20)
    results.append(evaluate_pipeline_unified(
        emb, hashes, pd, pivots, itq_model,
        'D', 'ITQ+Band(8)+Pivot(20)',
        use_band=True, band_width=8,
        use_pivot=True, pivot_threshold=20,
        candidate_limit=candidate_limit
    ))
    
    # D2: ITQ + Band + Pivot (bw=8, pt=25)
    results.append(evaluate_pipeline_unified(
        emb, hashes, pd, pivots, itq_model,
        'D2', 'ITQ+Band(8)+Pivot(25)',
        use_band=True, band_width=8,
        use_pivot=True, pivot_threshold=25,
        candidate_limit=candidate_limit
    ))
    
    # E: ITQ + Confidence probe (bw=8, 8 probes)
    results.append(evaluate_pipeline_unified(
        emb, hashes, pd, pivots, itq_model,
        'E', 'ITQ+Conf(bw=8,p=8)',
        use_confidence=True, band_width=8, max_probes=8,
        candidate_limit=candidate_limit
    ))
    
    # E2: ITQ + Confidence probe (bw=8, 16 probes)
    results.append(evaluate_pipeline_unified(
        emb, hashes, pd, pivots, itq_model,
        'E2', 'ITQ+Conf(bw=8,p=16)',
        use_confidence=True, band_width=8, max_probes=16,
        candidate_limit=candidate_limit
    ))
    
    # F: ITQ + Confidence + Pivot (bw=8, 8 probes, pt=20)
    results.append(evaluate_pipeline_unified(
        emb, hashes, pd, pivots, itq_model,
        'F', 'ITQ+Conf(8,8)+Pvt(20)',
        use_confidence=True, band_width=8, max_probes=8,
        use_pivot=True, pivot_threshold=20,
        candidate_limit=candidate_limit
    ))
    
    # F2: ITQ + Confidence + Pivot (bw=8, 16 probes, pt=25)
    results.append(evaluate_pipeline_unified(
        emb, hashes, pd, pivots, itq_model,
        'F2', 'ITQ+Conf(8,16)+Pvt(25)',
        use_confidence=True, band_width=8, max_probes=16,
        use_pivot=True, pivot_threshold=25,
        candidate_limit=candidate_limit
    ))
    
    return results


def print_results_table(results, dataset_name, candidate_limit):
    """結果テーブルを表示"""
    print(f'\n{dataset_name} (candidate_limit={candidate_limit})')
    print(f'{"ID":<4} {"Pipeline":<25} {"Cands":>8} {"Reduction":>10} '
          f'{"FiltRcl":>8} {"R@10":>8} {"Time(ms)":>10}')
    print('-' * 80)
    for r in results:
        print(f'{r["id"]:<4} {r["name"]:<25} {r["filter_candidates"]:>7.0f} '
              f'{r["reduction"]*100:>9.1f}% '
              f'{r["filter_recall"]*100:>7.1f}% '
              f'{r["recall_at_k"]*100:>7.1f}% '
              f'{r["time_ms"]:>9.2f}')

In [5]:
print('='*80)
print('Full Pipeline Comparison')
print('='*80)

all_results = {}

for dataset_name in ['EN', 'JA', 'MiniLM']:
    print(f'\nEvaluating {dataset_name}...')
    
    # candidate_limit=500
    results_500 = run_all_pipelines(datasets[dataset_name], dataset_name, candidate_limit=500)
    print_results_table(results_500, f'{dataset_name} E5-base' if dataset_name != 'MiniLM' else 'MiniLM', 500)
    all_results[f'{dataset_name}_500'] = results_500
    
    # candidate_limit=100
    results_100 = run_all_pipelines(datasets[dataset_name], dataset_name, candidate_limit=100)
    print_results_table(results_100, f'{dataset_name} E5-base' if dataset_name != 'MiniLM' else 'MiniLM', 100)
    all_results[f'{dataset_name}_100'] = results_100

Full Pipeline Comparison

Evaluating EN...



EN E5-base (candidate_limit=500)
ID   Pipeline                     Cands  Reduction  FiltRcl     R@10   Time(ms)
--------------------------------------------------------------------------------
A    ITQ Baseline                 9999       0.0%   100.0%    84.0%      3.28
B    ITQ+Pivot(t=20)              9055       9.4%    99.2%    84.2%      3.55
B2   ITQ+Pivot(t=25)              9825       1.7%    99.9%    84.0%      3.67
C    ITQ+Band(bw=8)               2181      78.2%    68.9%    66.0%      2.69
D    ITQ+Band(8)+Pivot(20)        2039      79.6%    68.4%    66.1%      2.75
D2   ITQ+Band(8)+Pivot(25)        2158      78.4%    68.8%    66.0%      2.84
E    ITQ+Conf(bw=8,p=8)           2793      72.1%    75.6%    71.4%      3.07
E2   ITQ+Conf(bw=8,p=16)          3757      62.4%    85.1%    78.0%      3.60
F    ITQ+Conf(8,8)+Pvt(20)        2601      74.0%    75.1%    71.0%      3.27
F2   ITQ+Conf(8,16)+Pvt(25)       3713      62.9%    85.0%    77.8%      3.75



EN E5-base (candidate_limit=100)
ID   Pipeline                     Cands  Reduction  FiltRcl     R@10   Time(ms)
--------------------------------------------------------------------------------
A    ITQ Baseline                 9999       0.0%   100.0%    61.0%      2.79
B    ITQ+Pivot(t=20)              9055       9.4%    99.2%    61.4%      3.05
B2   ITQ+Pivot(t=25)              9825       1.7%    99.9%    60.6%      3.17
C    ITQ+Band(bw=8)               2181      78.2%    68.9%    53.1%      1.88
D    ITQ+Band(8)+Pivot(20)        2039      79.6%    68.4%    53.6%      2.07
D2   ITQ+Band(8)+Pivot(25)        2158      78.4%    68.8%    53.4%      2.10
E    ITQ+Conf(bw=8,p=8)           2793      72.1%    75.6%    56.4%      2.40
E2   ITQ+Conf(bw=8,p=16)          3757      62.4%    85.1%    59.7%      2.84
F    ITQ+Conf(8,8)+Pvt(20)        2601      74.0%    75.1%    56.4%      2.58
F2   ITQ+Conf(8,16)+Pvt(25)       3713      62.9%    85.0%    59.6%      3.14

Evaluating JA...



JA E5-base (candidate_limit=500)
ID   Pipeline                     Cands  Reduction  FiltRcl     R@10   Time(ms)
--------------------------------------------------------------------------------
A    ITQ Baseline                 9999       0.0%   100.0%    98.1%      3.47
B    ITQ+Pivot(t=20)              6956      30.4%    98.1%    96.7%      3.43
B2   ITQ+Pivot(t=25)              8738      12.6%    99.9%    98.0%      3.69
C    ITQ+Band(bw=8)                694      93.1%    63.9%    63.9%      2.06
D    ITQ+Band(8)+Pivot(20)         560      94.4%    63.0%    63.0%      2.04
D2   ITQ+Band(8)+Pivot(25)         646      93.5%    63.8%    63.8%      2.17
E    ITQ+Conf(bw=8,p=8)           1002      90.0%    72.4%    72.4%      2.39
E2   ITQ+Conf(bw=8,p=16)          1318      86.8%    82.9%    82.7%      2.59
F    ITQ+Conf(8,8)+Pvt(20)         798      92.0%    71.5%    71.5%      2.46
F2   ITQ+Conf(8,16)+Pvt(25)       1221      87.8%    82.8%    82.6%      2.72



JA E5-base (candidate_limit=100)
ID   Pipeline                     Cands  Reduction  FiltRcl     R@10   Time(ms)
--------------------------------------------------------------------------------
A    ITQ Baseline                 9999       0.0%   100.0%    83.5%      2.84
B    ITQ+Pivot(t=20)              6956      30.4%    98.1%    81.8%      2.74
B2   ITQ+Pivot(t=25)              8738      12.6%    99.9%    83.1%      3.02
C    ITQ+Band(bw=8)                694      93.1%    63.9%    61.9%      1.29
D    ITQ+Band(8)+Pivot(20)         560      94.4%    63.0%    61.1%      1.39
D2   ITQ+Band(8)+Pivot(25)         646      93.5%    63.8%    61.7%      1.46
E    ITQ+Conf(bw=8,p=8)           1002      90.0%    72.4%    69.0%      1.69
E2   ITQ+Conf(bw=8,p=16)          1318      86.8%    82.9%    77.4%      1.90
F    ITQ+Conf(8,8)+Pvt(20)         798      92.0%    71.5%    68.7%      1.77
F2   ITQ+Conf(8,16)+Pvt(25)       1221      87.8%    82.8%    76.9%      2.03

Evaluating MiniLM...



MiniLM (candidate_limit=500)
ID   Pipeline                     Cands  Reduction  FiltRcl     R@10   Time(ms)
--------------------------------------------------------------------------------
A    ITQ Baseline                 9999       0.0%   100.0%    98.3%      3.36
B    ITQ+Pivot(t=20)              8025      19.8%    96.7%    95.2%      3.49
B2   ITQ+Pivot(t=25)              9360       6.4%    98.8%    97.1%      3.65
C    ITQ+Band(bw=8)                654      93.5%    50.9%    50.9%      1.90
D    ITQ+Band(8)+Pivot(20)         557      94.4%    49.2%    49.2%      1.96
D2   ITQ+Band(8)+Pivot(25)         624      93.8%    50.1%    50.1%      2.00
E    ITQ+Conf(bw=8,p=8)            955      90.5%    59.5%    59.5%      2.27
E2   ITQ+Conf(bw=8,p=16)          1250      87.5%    71.4%    71.4%      2.48
F    ITQ+Conf(8,8)+Pvt(20)         810      91.9%    57.5%    57.5%      2.35
F2   ITQ+Conf(8,16)+Pvt(25)       1192      88.1%    70.3%    70.3%      2.61



MiniLM (candidate_limit=100)
ID   Pipeline                     Cands  Reduction  FiltRcl     R@10   Time(ms)
--------------------------------------------------------------------------------
A    ITQ Baseline                 9999       0.0%   100.0%    87.7%      2.81
B    ITQ+Pivot(t=20)              8025      19.8%    96.7%    85.7%      2.88
B2   ITQ+Pivot(t=25)              9360       6.4%    98.8%    87.1%      3.10
C    ITQ+Band(bw=8)                654      93.5%    50.9%    50.5%      1.25
D    ITQ+Band(8)+Pivot(20)         557      94.4%    49.2%    48.8%      1.35
D2   ITQ+Band(8)+Pivot(25)         624      93.8%    50.1%    49.7%      1.38
E    ITQ+Conf(bw=8,p=8)            955      90.5%    59.5%    58.3%      1.65
E2   ITQ+Conf(bw=8,p=16)          1250      87.5%    71.4%    68.9%      1.92
F    ITQ+Conf(8,8)+Pvt(20)         810      91.9%    57.5%    56.5%      1.74
F2   ITQ+Conf(8,16)+Pvt(25)       1192      88.1%    70.3%    67.8%      2.01


## 4. Pareto最適フロンティア分析

In [6]:
print('='*80)
print('Pareto Optimal Frontier Analysis')
print('='*80)

def find_pareto(results):
    """Pareto最適解を特定（Recall最大化 & 候補数最小化）"""
    pareto = []
    for r in results:
        dominated = False
        for other in results:
            if other is r:
                continue
            # otherがrを支配: Recall同等以上 かつ 候補数同等以下 かつ 少なくとも1つで厳密に良い
            if (other['recall_at_k'] >= r['recall_at_k'] and 
                other['filter_candidates'] <= r['filter_candidates'] and
                (other['recall_at_k'] > r['recall_at_k'] or 
                 other['filter_candidates'] < r['filter_candidates'])):
                dominated = True
                break
        if not dominated:
            pareto.append(r)
    return sorted(pareto, key=lambda x: x['recall_at_k'], reverse=True)


for key in ['EN_500', 'JA_500', 'MiniLM_500']:
    results = all_results[key]
    pareto = find_pareto(results)
    
    dataset_label = key.split('_')[0]
    print(f'\n--- {dataset_label} (candidate_limit=500) ---')
    print(f'Pareto optimal pipelines ({len(pareto)}/{len(results)}):')
    for r in pareto:
        print(f'  {r["id"]}: {r["name"]:<25} R@10={r["recall_at_k"]*100:.1f}%, '
              f'Cands={r["filter_candidates"]:.0f}, Time={r["time_ms"]:.2f}ms')

Pareto Optimal Frontier Analysis

--- EN (candidate_limit=500) ---
Pareto optimal pipelines (6/10):
  B: ITQ+Pivot(t=20)           R@10=84.2%, Cands=9055, Time=3.55ms
  E2: ITQ+Conf(bw=8,p=16)       R@10=78.0%, Cands=3757, Time=3.60ms
  F2: ITQ+Conf(8,16)+Pvt(25)    R@10=77.8%, Cands=3713, Time=3.75ms
  E: ITQ+Conf(bw=8,p=8)        R@10=71.4%, Cands=2793, Time=3.07ms
  F: ITQ+Conf(8,8)+Pvt(20)     R@10=71.0%, Cands=2601, Time=3.27ms
  D: ITQ+Band(8)+Pivot(20)     R@10=66.1%, Cands=2039, Time=2.75ms

--- JA (candidate_limit=500) ---
Pareto optimal pipelines (10/10):
  A: ITQ Baseline              R@10=98.1%, Cands=9999, Time=3.47ms
  B2: ITQ+Pivot(t=25)           R@10=98.0%, Cands=8738, Time=3.69ms
  B: ITQ+Pivot(t=20)           R@10=96.7%, Cands=6956, Time=3.43ms
  E2: ITQ+Conf(bw=8,p=16)       R@10=82.7%, Cands=1318, Time=2.59ms
  F2: ITQ+Conf(8,16)+Pvt(25)    R@10=82.6%, Cands=1221, Time=2.72ms
  E: ITQ+Conf(bw=8,p=8)        R@10=72.4%, Cands=1002, Time=2.39ms
  F: ITQ+Conf(8,8)+Pvt(

## 5. 英語 vs 日本語の比較

In [7]:
print('='*80)
print('English vs Japanese Comparison')
print('='*80)

en_results = all_results['EN_500']
ja_results = all_results['JA_500']

print(f'\n{"Pipeline":<25} | {"--- English ---":^22} | {"--- Japanese ---":^22} | {"差":>6}')
print(f'{"":<25} | {"R@10":>8} {"Cands":>10} | {"R@10":>8} {"Cands":>10} | {"ΔR@10":>6}')
print('-' * 85)
for en, ja in zip(en_results, ja_results):
    delta = ja['recall_at_k'] * 100 - en['recall_at_k'] * 100
    print(f'{en["name"]:<25} | '
          f'{en["recall_at_k"]*100:>7.1f}% {en["filter_candidates"]:>9.0f} | '
          f'{ja["recall_at_k"]*100:>7.1f}% {ja["filter_candidates"]:>9.0f} | '
          f'{delta:>+5.1f}%')

English vs Japanese Comparison

Pipeline                  |    --- English ---     |    --- Japanese ---    |      差
                          |     R@10      Cands |     R@10      Cands |  ΔR@10
-------------------------------------------------------------------------------------
ITQ Baseline              |    84.0%      9999 |    98.1%      9999 | +14.1%
ITQ+Pivot(t=20)           |    84.2%      9055 |    96.7%      6956 | +12.5%
ITQ+Pivot(t=25)           |    84.0%      9825 |    98.0%      8738 | +14.0%
ITQ+Band(bw=8)            |    66.0%      2181 |    63.9%       694 |  -2.1%
ITQ+Band(8)+Pivot(20)     |    66.1%      2039 |    63.0%       560 |  -3.1%
ITQ+Band(8)+Pivot(25)     |    66.0%      2158 |    63.8%       646 |  -2.2%
ITQ+Conf(bw=8,p=8)        |    71.4%      2793 |    72.4%      1002 |  +1.0%
ITQ+Conf(bw=8,p=16)       |    78.0%      3757 |    82.7%      1318 |  +4.7%
ITQ+Conf(8,8)+Pvt(20)     |    71.0%      2601 |    71.5%       798 |  +0.5%
ITQ+Conf(8,16)+Pvt(25)   

## 6. 推奨パイプライン分析

In [8]:
print('='*80)
print('Recommended Pipeline Analysis')
print('='*80)

# 各データセットで最高Recall@10を達成するパイプラインを特定
for key in ['EN_500', 'JA_500', 'MiniLM_500']:
    results = all_results[key]
    dataset_label = key.split('_')[0]
    
    # Recall@10でソート
    sorted_results = sorted(results, key=lambda x: x['recall_at_k'], reverse=True)
    
    print(f'\n--- {dataset_label} ---')
    print(f'Top-3 by Recall@10:')
    for i, r in enumerate(sorted_results[:3]):
        print(f'  {i+1}. {r["name"]:<25} R@10={r["recall_at_k"]*100:.1f}%, '
              f'Cands={r["filter_candidates"]:.0f}, '
              f'Reduction={r["reduction"]*100:.1f}%, '
              f'Time={r["time_ms"]:.2f}ms')
    
    # 削減率50%以上でのベスト
    filtered = [r for r in results if r['reduction'] > 0.5]
    if filtered:
        best_filtered = max(filtered, key=lambda x: x['recall_at_k'])
        print(f'  Best with >50% reduction: {best_filtered["name"]} '
              f'R@10={best_filtered["recall_at_k"]*100:.1f}%, '
              f'Reduction={best_filtered["reduction"]*100:.1f}%')
    
    # 削減率90%以上でのベスト
    filtered90 = [r for r in results if r['reduction'] > 0.9]
    if filtered90:
        best_filtered90 = max(filtered90, key=lambda x: x['recall_at_k'])
        print(f'  Best with >90% reduction: {best_filtered90["name"]} '
              f'R@10={best_filtered90["recall_at_k"]*100:.1f}%, '
              f'Reduction={best_filtered90["reduction"]*100:.1f}%')

Recommended Pipeline Analysis

--- EN ---
Top-3 by Recall@10:
  1. ITQ+Pivot(t=20)           R@10=84.2%, Cands=9055, Reduction=9.4%, Time=3.55ms
  2. ITQ Baseline              R@10=84.0%, Cands=9999, Reduction=0.0%, Time=3.28ms
  3. ITQ+Pivot(t=25)           R@10=84.0%, Cands=9825, Reduction=1.7%, Time=3.67ms
  Best with >50% reduction: ITQ+Conf(bw=8,p=16) R@10=78.0%, Reduction=62.4%

--- JA ---
Top-3 by Recall@10:
  1. ITQ Baseline              R@10=98.1%, Cands=9999, Reduction=0.0%, Time=3.47ms
  2. ITQ+Pivot(t=25)           R@10=98.0%, Cands=8738, Reduction=12.6%, Time=3.69ms
  3. ITQ+Pivot(t=20)           R@10=96.7%, Cands=6956, Reduction=30.4%, Time=3.43ms
  Best with >50% reduction: ITQ+Conf(bw=8,p=16) R@10=82.7%, Reduction=86.8%
  Best with >90% reduction: ITQ+Conf(8,8)+Pvt(20) R@10=71.5%, Reduction=92.0%

--- MiniLM ---
Top-3 by Recall@10:
  1. ITQ Baseline              R@10=98.3%, Cands=9999, Reduction=0.0%, Time=3.36ms
  2. ITQ+Pivot(t=25)           R@10=97.1%, Cands=9360, Re

## 7. Filter Recall vs Recall@10の関係

In [9]:
print('='*80)
print('Filter Recall vs Recall@10 Analysis')
print('='*80)

print('\nFilter Recallが低い場合のR@10への影響:')
print('（Filter Recallは真のTop-10が候補に残る割合、R@10は最終精度）')

for key in ['EN_500', 'JA_500']:
    results = all_results[key]
    dataset_label = key.split('_')[0]
    
    print(f'\n--- {dataset_label} ---')
    print(f'{"Pipeline":<25} {"FiltRcl":>8} {"R@10":>8} {"Gap":>8} {"Cands":>8}')
    print('-' * 60)
    for r in results:
        gap = r['filter_recall'] * 100 - r['recall_at_k'] * 100
        print(f'{r["name"]:<25} {r["filter_recall"]*100:>7.1f}% '
              f'{r["recall_at_k"]*100:>7.1f}% {gap:>+7.1f}% '
              f'{r["filter_candidates"]:>7.0f}')

Filter Recall vs Recall@10 Analysis

Filter Recallが低い場合のR@10への影響:
（Filter Recallは真のTop-10が候補に残る割合、R@10は最終精度）

--- EN ---
Pipeline                   FiltRcl     R@10      Gap    Cands
------------------------------------------------------------
ITQ Baseline                100.0%    84.0%   +16.0%    9999
ITQ+Pivot(t=20)              99.2%    84.2%   +15.0%    9055
ITQ+Pivot(t=25)              99.9%    84.0%   +15.9%    9825
ITQ+Band(bw=8)               68.9%    66.0%    +2.9%    2181
ITQ+Band(8)+Pivot(20)        68.4%    66.1%    +2.3%    2039
ITQ+Band(8)+Pivot(25)        68.8%    66.0%    +2.8%    2158
ITQ+Conf(bw=8,p=8)           75.6%    71.4%    +4.2%    2793
ITQ+Conf(bw=8,p=16)          85.1%    78.0%    +7.1%    3757
ITQ+Conf(8,8)+Pvt(20)        75.1%    71.0%    +4.1%    2601
ITQ+Conf(8,16)+Pvt(25)       85.0%    77.8%    +7.2%    3713

--- JA ---
Pipeline                   FiltRcl     R@10      Gap    Cands
------------------------------------------------------------
ITQ Baselin

## 8. 総合サマリー

In [10]:
print('='*80)
print('DF-LSH Integration Experiment Series (80-84) Summary')
print('='*80)

print('\n【実験81: DF-LSH独立評価】')
print('- ITQ-LSHのバイナリコード品質がDF-LSH(PCA)を大幅に上回る')
print('  (Spearman: ITQ=-0.47 vs PCA=-0.23 / English)')
print('- ITQ回転行列の最適化が量子化品質に大きく寄与')
print('- DF-LSH独立では既存ITQ-LSHを覆す効果なし')

print('\n【実験82: Bloomフィルタ（バンドプリフィルタ）】')
print('- バンドフィルタ（bw=8）でITQハッシュの候補を高速絞り込み可能')
print('- Pivotと組み合わせることで2段フィルタリングが機能')
print('- ただし、バンド完全一致は厳しく、Recall低下のリスク')

print('\n【実験83: Confidence Multi-probe】')
print('- ITQ射影値の|Z|を確信度として利用したmulti-probe')
print('- confidence順 vs ランダム順でprobe効率を比較')
print('- 確信度の低いビットを優先フリップすることでRecall回復')

print('\n【実験84: 総合比較（本ノートブック）】')

# 各データセットの推奨を出力
for key in ['EN_500', 'JA_500', 'MiniLM_500']:
    results = all_results[key]
    dataset_label = key.split('_')[0]
    
    baseline = results[0]  # A: ITQ Baseline
    best = max(results, key=lambda x: x['recall_at_k'])
    best_efficient = max(
        [r for r in results if r['reduction'] > 0.5],
        key=lambda x: x['recall_at_k'],
        default=None
    )
    
    print(f'\n  [{dataset_label}]')
    print(f'    Baseline (A):  R@10={baseline["recall_at_k"]*100:.1f}%')
    print(f'    Best overall:  {best["name"]} R@10={best["recall_at_k"]*100:.1f}%')
    if best_efficient:
        print(f'    Best efficient: {best_efficient["name"]} '
              f'R@10={best_efficient["recall_at_k"]*100:.1f}%, '
              f'Reduction={best_efficient["reduction"]*100:.1f}%')

print('\n【結論】')
print('1. ITQ回転行列の最適化は量子化品質の核心であり、DF-LSHのPCA射影では代替不可')
print('2. バンドインデックスはITQハッシュ上でプリフィルタとして利用可能')
print('3. Confidence multi-probeにより、バンドフィルタのRecall低下を部分的に回復')
print('4. 現行のITQ+Pivotパイプラインが依然として最良のバランス')
print('5. バンドプリフィルタの追加は、大規模データ（100K+）で計算量削減の効果が期待')

DF-LSH Integration Experiment Series (80-84) Summary

【実験81: DF-LSH独立評価】
- ITQ-LSHのバイナリコード品質がDF-LSH(PCA)を大幅に上回る
  (Spearman: ITQ=-0.47 vs PCA=-0.23 / English)
- ITQ回転行列の最適化が量子化品質に大きく寄与
- DF-LSH独立では既存ITQ-LSHを覆す効果なし

【実験82: Bloomフィルタ（バンドプリフィルタ）】
- バンドフィルタ（bw=8）でITQハッシュの候補を高速絞り込み可能
- Pivotと組み合わせることで2段フィルタリングが機能
- ただし、バンド完全一致は厳しく、Recall低下のリスク

【実験83: Confidence Multi-probe】
- ITQ射影値の|Z|を確信度として利用したmulti-probe
- confidence順 vs ランダム順でprobe効率を比較
- 確信度の低いビットを優先フリップすることでRecall回復

【実験84: 総合比較（本ノートブック）】

  [EN]
    Baseline (A):  R@10=84.0%
    Best overall:  ITQ+Pivot(t=20) R@10=84.2%
    Best efficient: ITQ+Conf(bw=8,p=16) R@10=78.0%, Reduction=62.4%

  [JA]
    Baseline (A):  R@10=98.1%
    Best overall:  ITQ Baseline R@10=98.1%
    Best efficient: ITQ+Conf(bw=8,p=16) R@10=82.7%, Reduction=86.8%

  [MiniLM]
    Baseline (A):  R@10=98.3%
    Best overall:  ITQ Baseline R@10=98.3%
    Best efficient: ITQ+Conf(bw=8,p=16) R@10=71.4%, Reduction=87.5%

【結論】
1. ITQ回転行列の最適化は量子化品質の核心であり、DF-LSHのPCA射影では代替